In [1]:
import os
import shutil
import stat
import seaborn as sns
import collections
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.image as img
import random
import cv2
import PIL
import matplotlib.pyplot as plt
import matplotlib.image as img
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from sklearn.model_selection import train_test_split
from skimage.io import imread
from keras.utils.np_utils import to_categorical
from keras.applications.inception_v3 import preprocess_input
from keras.models import load_model
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models
import pandas as pd

In [2]:
def split_path(data, mode='id'):
    if mode == 'class':
        return data.split('/')[0]
    return data.split('/')[-1]

def format_path(path):
    return f'E:/food-101/images/{path}.jpg'

In [3]:
import pandas as pd

# Load dataset
train = pd.read_csv(r'E:\food-101\meta\train.txt', header=None, names=['path'])
test = pd.read_csv(r'E:\food-101\meta\test.txt', header=None, names=['path'])


In [4]:
# Process data
train['label'] = train['path'].map(lambda x: split_path(x, 'class')) 
train['idx'] = train['path'].map(split_path) 
test['label'] = test['path'].map(lambda x: split_path(x, 'class')) 
test['idx'] = test['path'].map(split_path) 

# Tüm sınıfları içeren bir listeyi oluştur
classes = train['label'].unique()

# Veri yolu formatını güncelle
train['path'] = train['path'].apply(format_path)
test['path'] = test['path'].apply(format_path)


In [5]:
import os
import shutil

def copy_files(data, destination):
    # Verilen veri kümesindeki dosyaları ilgili sınıflara göre kopyalar
    for index, row in data.iterrows():
        src = row['path']
        label = row['label']
        class_folder = os.path.join(destination, label)
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)
        dst = os.path.join(class_folder, row['idx'] + '.jpg')
        shutil.copy(src, dst)

def create_folders(path):
    # Verilen yol altında sınıf adlarıyla klasörler oluşturur
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        print("Klasör zaten var!")

# Train ve Test için klasörler oluşturalım
train_data_path = 'E:/food-101/train'
test_data_path = 'E:/food-101/test'

create_folders(train_data_path)
create_folders(test_data_path)

# Dosyaları kopyalayalım
copy_files(train, train_data_path)
copy_files(test, test_data_path)

In [6]:
def count_images_in_classes(folder_path):
    class_counts = {}
    for class_name in os.listdir(folder_path):
        class_dir = os.path.join(folder_path, class_name)
        if os.path.isdir(class_dir):
            num_images = len([file for file in os.listdir(class_dir) if file.endswith('.jpg')])
            class_counts[class_name] = num_images
    return class_counts

# Train ve test klasörlerindeki sınıf adlarını ve resim sayılarını yazdıralım
train_class_counts = count_images_in_classes(train_data_path)
test_class_counts = count_images_in_classes(test_data_path)

print("Train Klasöründeki Sınıflar ve Resim Sayıları:")
for class_name, count in train_class_counts.items():
    print(f"{class_name}: {count} resim")

print("\nTest Klasöründeki Sınıflar ve Resim Sayıları:")
for class_name, count in test_class_counts.items():
    print(f"{class_name}: {count} resim")


Train Klasöründeki Sınıflar ve Resim Sayıları:
apple_pie: 750 resim
baby_back_ribs: 750 resim
baklava: 750 resim
beef_carpaccio: 750 resim
beef_tartare: 750 resim
beet_salad: 750 resim
beignets: 750 resim
bibimbap: 750 resim
bread_pudding: 750 resim
breakfast_burrito: 750 resim
bruschetta: 750 resim
caesar_salad: 750 resim
cannoli: 750 resim
caprese_salad: 750 resim
carrot_cake: 750 resim
ceviche: 750 resim
cheesecake: 750 resim
cheese_plate: 750 resim
chicken_curry: 750 resim
chicken_quesadilla: 750 resim
chicken_wings: 750 resim
chocolate_cake: 750 resim
chocolate_mousse: 750 resim
churros: 750 resim
clam_chowder: 750 resim
club_sandwich: 750 resim
crab_cakes: 750 resim
creme_brulee: 750 resim
croque_madame: 750 resim
cup_cakes: 750 resim
deviled_eggs: 750 resim
donuts: 750 resim
dumplings: 750 resim
edamame: 750 resim
eggs_benedict: 750 resim
escargots: 750 resim
falafel: 750 resim
filet_mignon: 750 resim
fish_and_chips: 750 resim
foie_gras: 750 resim
french_fries: 750 resim
french_

In [7]:
def find_duplicate_images(train_folder, test_folder):
    duplicate_found = False

    for class_name in os.listdir(train_folder):
        train_class_dir = os.path.join(train_folder, class_name)
        test_class_dir = os.path.join(test_folder, class_name)

        if os.path.isdir(train_class_dir) and os.path.isdir(test_class_dir):
            train_images = set(os.listdir(train_class_dir))
            test_images = set(os.listdir(test_class_dir))

            common_images = train_images.intersection(test_images)
            for image_file in common_images:
                train_image_path = os.path.join(train_class_dir, image_file)
                test_image_path = os.path.join(test_class_dir, image_file)
                if compare_images(train_image_path, test_image_path):
                    duplicate_found = True
                    break

    if duplicate_found:
        print("Train ve test klasörleri arasında aynı içeriğe sahip resim dosyaları bulundu.")
    else:
        print("Train ve test klasörleri arasında aynı içeriğe sahip resim dosyası bulunamadı.")

# Train ve test klasörleri arasında yinelenen resim dosyalarını kontrol edelim
find_duplicate_images(train_data_path, test_data_path)


Train ve test klasörleri arasında aynı içeriğe sahip resim dosyası bulunamadı.


In [8]:
def get_first_image_per_class(folder_path):
    first_images = []

    # Klasördeki sınıfları sıralayalım
    classes = sorted(os.listdir(folder_path))

    # Her sınıfın ilk resmini alalım
    for class_name in classes:
        class_dir = os.path.join(folder_path, class_name)
        if os.path.isdir(class_dir):
            images = os.listdir(class_dir)
            if images:  # Eğer sınıf klasöründe resim varsa
                image_path = os.path.join(class_dir, images[0])
                first_images.append(image_path)

    return first_images

# Train klasöründeki ilk sınıfların ilk resimlerini alalım
train_first_images = get_first_image_per_class(train_data_path)

# Test klasöründeki ilk sınıfların ilk resimlerini alalım
test_first_images = get_first_image_per_class(test_data_path)

# Sonuçları ekrana yazdıralım
print("Train klasöründeki ilk sınıfların ilk resimleri:")
for idx, image_path in enumerate(train_first_images, start=1):
    print(f"{idx}. resim: {image_path}")

print("\nTest klasöründeki ilk sınıfların ilk resimleri:")
for idx, image_path in enumerate(test_first_images, start=1):
    print(f"{idx}. resim: {image_path}")


Train klasöründeki ilk sınıfların ilk resimleri:
1. resim: E:/food-101/train\apple_pie\1005649.jpg
2. resim: E:/food-101/train\baby_back_ribs\1005293.jpg
3. resim: E:/food-101/train\baklava\1006121.jpg
4. resim: E:/food-101/train\beef_carpaccio\1011469.jpg
5. resim: E:/food-101/train\beef_tartare\1000431.jpg
6. resim: E:/food-101/train\beet_salad\1003501.jpg
7. resim: E:/food-101/train\beignets\1000911.jpg
8. resim: E:/food-101/train\bibimbap\1002297.jpg
9. resim: E:/food-101/train\bread_pudding\1004499.jpg
10. resim: E:/food-101/train\breakfast_burrito\1000351.jpg
11. resim: E:/food-101/train\bruschetta\1003796.jpg
12. resim: E:/food-101/train\caesar_salad\1000016.jpg
13. resim: E:/food-101/train\cannoli\1007970.jpg
14. resim: E:/food-101/train\caprese_salad\1001946.jpg
15. resim: E:/food-101/train\carrot_cake\1000447.jpg
16. resim: E:/food-101/train\ceviche\1006106.jpg
17. resim: E:/food-101/train\cheese_plate\100370.jpg
18. resim: E:/food-101/train\cheesecake\1001446.jpg
19. resim: 

In [ ]:
def count_classes(folder_path):
    num_classes = len([name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))])
    return num_classes

# Train klasöründeki sınıf sayısını alalım
num_train_classes = count_classes(train_data_path)

# Test klasöründeki sınıf sayısını alalım
num_test_classes = count_classes(test_data_path)

print(f"Train klasöründeki sınıf sayısı: {num_train_classes}")
print(f"Test klasöründeki sınıf sayısı: {num_test_classes}")


In [9]:
import os

def count_images_in_folder(folder_path):
    total_images = 0
    for root, dirs, files in os.walk(folder_path):
        total_images += len(files)
    return total_images

# Train ve test klasörlerindeki toplam resim sayısını alalım
total_train_images = count_images_in_folder(train_data_path)
total_test_images = count_images_in_folder(test_data_path)

print("Train klasöründeki toplam resim sayısı:", total_train_images)
print("Test klasöründeki toplam resim sayısı:", total_test_images)


Train klasöründeki toplam resim sayısı: 75750
Test klasöründeki toplam resim sayısı: 25250


In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K

# Clear Keras session
K.clear_session()

# Define constants
n_classes = 101
batch_size = 16
width, height = 224, 224
train_data_path = r"E:\food-101\train"
test_data_path = r"E:\food-101\test"
train_samples = 75750
test_samples = 25250

# Data generators
train_data_gen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest', rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, brightness_range=[0.6, 1.4])
test_data_gen = ImageDataGenerator(rescale=1. / 255)
train_gen = train_data_gen.flow_from_directory(train_data_path, target_size=(height, width), batch_size=batch_size, class_mode='categorical')
test_gen = test_data_gen.flow_from_directory(test_data_path, target_size=(height, width), batch_size=batch_size, class_mode='categorical')

# Function to create model
def create_model(learning_rate=0.0001, momentum=0.9, decay=1e-6, dropout_rate=0.2, l2_reg=0.005):
    # Base MobileNetV2 model
    mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(height, width, 3))
    layer = mobilenet.output
    layer = GlobalAveragePooling2D()(layer)
    layer = Dense(128, activation='relu')(layer)
    layer = Dropout(dropout_rate)(layer)
    predictions = Dense(n_classes, kernel_regularizer=regularizers.l2(l2_reg), activation='softmax')(layer)
    model = Model(inputs=mobilenet.input, outputs=predictions)

    # Compile model
    optimizer = SGD(learning_rate=learning_rate, momentum=momentum, decay=decay)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the model parameters
params = {'learning_rate': 0.001, 'momentum': 0.95, 'dropout_rate': 0.3, 'l2_reg': 0.005}

# Create model with specified parameters
model = create_model(**params)

# Load previous model weights if exists
checkpoint_path = "best_model_101class_mobilenetv2den5.hdf5"
if os.path.exists(checkpoint_path):
    model.load_weights(checkpoint_path)
    print("Loaded model weights from:", checkpoint_path)

# Define callbacks
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger('history_101class_mobilenetv2den5.log', append=True)

# Train model
history = model.fit(train_gen,
                    steps_per_epoch=train_samples // batch_size,
                    validation_data=test_gen,
                    validation_steps=test_samples // batch_size,
                    epochs=30,
                    callbacks=[checkpoint, csv_logger],
                    verbose=1)

# Write epoch-wise metrics to CSV file
epoch_metrics = pd.DataFrame({
    'Epoch': np.arange(1, 31),
    'Loss': history.history['loss'],
    'Accuracy': history.history['accuracy'],
    'Val Loss': history.history['val_loss'],
    'Val Accuracy': history.history['val_accuracy']
})
epoch_metrics.to_csv('epoch_metrics_mobilenetv2den5.csv', index=False)


Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.
Epoch 1/30
4734/4734 [==============================] - ETA: 0s - loss: 3.1833 - accuracy: 0.3138
Epoch 1: val_accuracy improved from -inf to 0.42570, saving model to best_model_101class_mobilenetv2den5.hdf5
4734/4734 [==============================] - 1114s 235ms/step - loss: 3.1833 - accuracy: 0.3138 - val_loss: 2.5211 - val_accuracy: 0.4257
Epoch 2/30
4734/4734 [==============================] - ETA: 0s - loss: 2.2743 - accuracy: 0.4749
Epoch 2: val_accuracy improved from 0.42570 to 0.46158, saving model to best_model_101class_mobilenetv2den5.hdf5
4734/4734 [==============================] - 827s 175ms/step - loss: 2.2743 - accuracy: 0.4749 - val_loss: 2.3771 - val_accuracy: 0.4616
Epoch 3/30
4734/4734 [==============================] - ETA: 0s - loss: 2.0409 - accuracy: 0.5297
Epoch 3: val_accuracy improved from 0.46158 to 0.48067, saving model to best_model_101class_mobilenetv2den5.hdf5
4734/

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Define constants
n_classes = 101
batch_size = 16
width, height = 224, 224
test_data_path = r"E:\food-101\test"
test_samples = 25250

# Load the best model
model_path = r"C:\Users\kurt_\best_model_101class_mobilenetv2den5.hdf5"
model = load_model(model_path)

# Data generator for test data
test_data_gen = ImageDataGenerator(rescale=1. / 255)
test_gen = test_data_gen.flow_from_directory(test_data_path, target_size=(height, width), batch_size=batch_size, class_mode='categorical', shuffle=False)

# Predict on test data
predictions = model.predict(test_gen, steps=test_gen.samples // test_gen.batch_size + 1, verbose=1)
y_pred = np.argmax(predictions, axis=1)

# Get the true labels
y_true = test_gen.classes

# Ensure the predictions and true labels have the same length
if len(y_pred) > len(y_true):
    y_pred = y_pred[:len(y_true)]
elif len(y_pred) < len(y_true):
    y_true = y_true[:len(y_pred)]

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Detailed classification report
report = classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys())
print(report)


Found 25250 images belonging to 101 classes.
1579/1579 [==============================] - 55s 35ms/step
Accuracy: 0.7703
Precision: 0.7926
Recall: 0.7703
F1 Score: 0.7715
                         precision    recall  f1-score   support

              apple_pie       0.51      0.64      0.57       250
         baby_back_ribs       0.82      0.79      0.80       250
                baklava       0.86      0.85      0.85       250
         beef_carpaccio       0.89      0.78      0.84       250
           beef_tartare       0.86      0.61      0.71       250
             beet_salad       0.94      0.50      0.65       250
               beignets       0.95      0.76      0.84       250
               bibimbap       0.94      0.84      0.89       250
          bread_pudding       0.83      0.38      0.52       250
      breakfast_burrito       0.66      0.67      0.66       250
             bruschetta       0.83      0.61      0.70       250
           caesar_salad       0.88      0.84    

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K

# Clear Keras session
K.clear_session()

# Define constants
n_classes = 101
batch_size = 16
width, height = 224, 224
train_data_path = r"E:\food-101\train"
test_data_path = r"E:\food-101\test"
train_samples = 75750
test_samples = 25250

# Data generators
train_data_gen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.6, 1.4]
)
test_data_gen = ImageDataGenerator(rescale=1. / 255)
train_gen = train_data_gen.flow_from_directory(
    train_data_path, 
    target_size=(height, width), 
    batch_size=batch_size, 
    class_mode='categorical'
)
test_gen = test_data_gen.flow_from_directory(
    test_data_path, 
    target_size=(height, width), 
    batch_size=batch_size, 
    class_mode='categorical'
)

# Function to create model
def create_model(learning_rate=0.0001, momentum=0.9, decay=1e-6, dropout_rate=0.2, l2_reg=0.005):
    # Base MobileNetV2 model
    mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(height, width, 3))
    
    # Freeze the base model
    for layer in mobilenet.layers:
        layer.trainable = False
    
    layer = mobilenet.output
    layer = GlobalAveragePooling2D()(layer)
    layer = Dense(128, activation='relu')(layer)
    layer = Dropout(dropout_rate)(layer)
    predictions = Dense(n_classes, kernel_regularizer=regularizers.l2(l2_reg), activation='softmax')(layer)
    model = Model(inputs=mobilenet.input, outputs=predictions)

    # Compile model
    optimizer = SGD(learning_rate=learning_rate, momentum=momentum, decay=decay)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the model parameters
params = {'learning_rate': 0.0001, 'momentum': 0.95, 'dropout_rate': 0.3, 'l2_reg': 0.005}

# Create model with specified parameters
model = create_model(**params)

# Load previous model weights if exists
checkpoint_path = r"C:\Users\kurt_\best_model_101class_mobilenetv2den5.hdf5"
if os.path.exists(checkpoint_path):
    model.load_weights(checkpoint_path)
    print("Loaded model weights from:", checkpoint_path)

# Unfreeze some layers for fine-tuning
for layer in model.layers[-20:]:  # Unfreeze the last 20 layers
    layer.trainable = True

# Compile the model again with a lower learning rate
optimizer = SGD(learning_rate=0.00001, momentum=0.95, decay=1e-6)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger('fine_tuning_history_101class_mobilenetv2den5.log', append=True)

# Continue training the model
history = model.fit(
    train_gen,
    steps_per_epoch=train_samples // batch_size,
    validation_data=test_gen,
    validation_steps=test_samples // batch_size,
    epochs=30,  # Increase this number if needed
    callbacks=[checkpoint, csv_logger],
    verbose=1
)

# Evaluate model
model.load_weights(checkpoint_path)
test_gen.reset()
pred = model.predict(test_gen, steps=test_samples // batch_size, verbose=1)
y_pred = np.argmax(pred, axis=1)
y_true = test_gen.classes[:len(y_pred)]

# Print classification report
print(classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys()))

# Save epoch-wise metrics to CSV file
epoch_metrics = pd.DataFrame({
    'Epoch': np.arange(1, len(history.history['loss']) + 1),
    'Loss': history.history['loss'],
    'Accuracy': history.history['accuracy'],
    'Val Loss': history.history['val_loss'],
    'Val Accuracy': history.history['val_accuracy']
})
epoch_metrics.to_csv('fine_tuning_epoch_metrics_mobilenetv2den5.csv', index=False)


Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.
Loaded model weights from: C:\Users\kurt_\best_model_101class_mobilenetv2den5.hdf5
Epoch 1/30
4734/4734 [==============================] - ETA: 0s - loss: 0.9677 - accuracy: 0.7829
Epoch 1: val_accuracy improved from -inf to 0.79392, saving model to C:\Users\kurt_\best_model_101class_mobilenetv2den5.hdf5
4734/4734 [==============================] - 1154s 243ms/step - loss: 0.9677 - accuracy: 0.7829 - val_loss: 0.9576 - val_accuracy: 0.7939
Epoch 2/30
4734/4734 [==============================] - ETA: 0s - loss: 0.9265 - accuracy: 0.7924
Epoch 2: val_accuracy improved from 0.79392 to 0.79626, saving model to C:\Users\kurt_\best_model_101class_mobilenetv2den5.hdf5
4734/4734 [==============================] - 1051s 222ms/step - loss: 0.9265 - accuracy: 0.7924 - val_loss: 0.9459 - val_accuracy: 0.7963
Epoch 3/30
4734/4734 [==============================] - ETA: 0s - loss: 0.9097 - accuracy: 0.7977
Epoc

KeyboardInterrupt: 

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Define constants
n_classes = 101
batch_size = 16
width, height = 224, 224
test_data_path = r"E:\food-101\test"
test_samples = 25250

# Load the best model
model_path = r"C:\Users\kurt_\best_model_101class_mobilenetv2den5.hdf5"
model = load_model(model_path)

# Data generator for test data
test_data_gen = ImageDataGenerator(rescale=1. / 255)
test_gen = test_data_gen.flow_from_directory(test_data_path, target_size=(height, width), batch_size=batch_size, class_mode='categorical', shuffle=False)

# Predict on test data
predictions = model.predict(test_gen, steps=test_gen.samples // test_gen.batch_size + 1, verbose=1)
y_pred = np.argmax(predictions, axis=1)

# Get the true labels
y_true = test_gen.classes

# Ensure the predictions and true labels have the same length
if len(y_pred) > len(y_true):
    y_pred = y_pred[:len(y_true)]
elif len(y_pred) < len(y_true):
    y_true = y_true[:len(y_pred)]

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Detailed classification report
report = classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys())
print(report)


Found 25250 images belonging to 101 classes.
1579/1579 [==============================] - 55s 34ms/step
Accuracy: 0.8064
Precision: 0.8106
Recall: 0.8064
F1 Score: 0.8065
                         precision    recall  f1-score   support

              apple_pie       0.62      0.59      0.61       250
         baby_back_ribs       0.74      0.84      0.79       250
                baklava       0.89      0.86      0.88       250
         beef_carpaccio       0.82      0.88      0.85       250
           beef_tartare       0.85      0.68      0.76       250
             beet_salad       0.77      0.74      0.76       250
               beignets       0.86      0.88      0.87       250
               bibimbap       0.95      0.88      0.92       250
          bread_pudding       0.63      0.58      0.61       250
      breakfast_burrito       0.75      0.68      0.72       250
             bruschetta       0.72      0.74      0.73       250
           caesar_salad       0.88      0.86    